# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Using the latest cached version of the dataset since yelp_review_full couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'yelp_review_full' at /home/chenglim/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/0.0.0/c1f9ee939b7d05667af864ee1cb066393154bf85 (last modified on Thu Feb  6 21:06:07 2025).


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [2]:
dataset["train"][11]

{'label': 0,
 'text': "This place is absolute garbage...  Half of the tees are not available, including all the grass tees.  It is cash only, and they sell the last bucket at 8, despite having lights.  And if you finish even a minute after 8, don't plan on getting a drink.  The vending machines are sold out (of course) and they sell drinks inside, but close the drawers at 8 on the dot.  There are weeds grown all over the place.  I noticed some sort of batting cage, but it looks like those are out of order as well.  Someone should buy this place and turn it into what it should be."}

In [3]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [4]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,"Not very impressed by the service and food. First, the skinny side of the food is loaded with sugars and toppings which are not healthy. For being a newer restaurant, you would think the service would be top notch to get the word out. Anyway, when we entered, I made it a point to say it was my first time because I did not understand if we should seat ourselves or order first. The lady was not friendly and could not answer detailed questions about the menu. The food was fair but the salad had heavy dressings. There is noting unique about this place. Meh..."
1,1 star,"This review is not about their food, we didn't even make it that far. The one star is for their policy against sharing. \n\nWe went to the Caesars for Restaurant week knowing there would be many restaurants there participating. My group consisted of 2 adults and a child. None of us were particularly hungry so we ordered one lunch off the Restaurant week menu and a mac and cheese. The waitress said that we could not share so I told her we were not going to share. After talking to her manager she came back to question what my boyfriend would be eating and after he responded that he would just be nibbling off mine she again told us there was a strict no sharing policy and that sharing would not be allowed. So we went to Bobby Flay's Mesa Grill instead where they had no policy against sharing.\n\nI was very disappointed with Max Brenner and do not understand the purpose of such policy. Isn't the primary point of Restaurant Week to support fighting hunger in Las Vegas? The policy feels like they are primarily using Restaurant week to help themselves. In addition, we went during brunch and the restaurant wasn't even busy."
2,1 star,"The reviews were very good so my wife and I tried this place last night. Granted it was the Friday before Christmas and it was 30 minutes before closing but we could not have been more disappointed. My wife ordered enchiladas and I had a carne asada burrito. Mine came to the table 5 minutes after my wife's and when I bit into it it was cold. I sent it back to the kitchen and 5 minutes later it was re-served cold again. Besides the temperature, the meat was very tough to cut. At that point we gave up after registering our complaint with the manager. He asked if I wanted something else but by this time I had no confidence in the kitchen. My wife's meal was fair at best. She said her food was tasteless and not very fresh. I'm afraid we won't be back. There are too many good restaurants in Scottsdale to risk another disappointment."
3,5 stars,"What an experience. What. An. Experience!\n\nI refuse to rehash the descriptions of desserts or the surprises in store at this establishment. I will just sum it up with a gorgeously white-with-red-accents appearance that you will just sink into a cloud of flavors and Miyazaki's music and find nirvana in a decadent dessert. It is an experience that I have to force myself not to go to TOO often as I don't ever want to ruin this hide away for myself. \n\nI had a moment where I took a bite of my dessert and paused - thanked whatever powers that be that I was able to eat such an amazing dish, that someone came up with this dessert (the brilliant Chef Mio), and that I was fortunate enough to sit in this amazing restaurant with this delectable confection, and for an hour or so - all was right with the world and nothing else matter. As my friend sitting next to me last night remarked halfway into our main dessert - \""I am just so happy right now.\"""
4,3 stars,"Interesting little place. Like all things Vegas, its pricey. I ordered a souvenir guitar yard cup which I know, I know that in itself runs the bills. I ordered a sample of their pina colada. I will tell you now I am a complete snob when it comes to my coladas. I want the real deal, so none of that crappy syrup sugar mix. Just, no. Of course, that's what they were serving so I ordered the Vacation. It was good but damn! (can I say that

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/home/chenglim/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"Sorry, I cannot give Al's Garage a good feedback. We were there Saturday evening. There was a free concert, and although it was crowded, we found a nice place to eat outside on the patio. We ordered chicken tenders with fries. They were alright, but nothing to write home about. \nWe were very patient, but after about 10 minutes, we asked if we could order. It took another 5 minutes until the waitress came to our table. She was very unfriendly and probably unhappy to work that evening. For whatever service we needed (napkins, refill etc) we had to battle to get her attention. She had no smile, or apology whatsoever... The people we shared the table with waited to pay for about 15 minutes. They were also not very satisfied...\nThere are too many good places around, so why bother and go back...","[101, 6502, 117, 146, 2834, 1660, 2586, 112, 188, 144, 4626, 2176, 170, 1363, 13032, 119, 1284, 1127, 1175, 4306, 3440, 119, 1247, 1108, 170, 1714, 3838, 117, 1105, 1780, 1122, 1108, 11090, 117, 1195, 1276, 170, 3505, 1282, 1106, 3940, 1796, 1113, 1103, 22408, 119, 1284, 2802, 9323, 8886, 1116, 1114, 175, 3377, 119, 1220, 1127, 15354, 117, 1133, 1720, 1106, 3593, 1313, 1164, 119, 165, 183, 2924, 1162, 1127, 1304, 5351, 117, 1133, 1170, 1164, 1275, 1904, 117, 1195, 1455, 1191, 1195, 1180, 1546, 119, 1135, 1261, 1330, 126, 1904, 1235, 1103, 15098, 1338, 1106, 1412, 1952, 119, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [9]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [ ]:
# 完整的超参数配置
print(training_args)

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [ ]:
import evaluate
evaluate.list_evaluation_modules()

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("./evaluate-main/metrics/accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=300)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
# 使用下面配置全量数据进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.765600,0.760925,0.666720
2,0.694100,0.730331,0.683200
3,0.580900,0.733673,0.690640


TrainOutput(global_step=121875, training_loss=0.7078223370517829, metrics={'train_runtime': 39693.9396, 'train_samples_per_second': 49.126, 'train_steps_per_second': 3.07, 'total_flos': 5.130803778048e+17, 'train_loss': 0.7078223370517829, 'epoch': 3.0})

In [26]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

尝试使用所有的测试集来进行验证，得到的准确率是0.69,比原来的0.53略高

In [29]:
# 尝试使用所有的测试集来进行验证，得到的准确率是0.69,比原来的0.53略高
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.7336730360984802,
 'eval_accuracy': 0.69064,
 'eval_runtime': 340.8857,
 'eval_samples_per_second': 146.677,
 'eval_steps_per_second': 18.335,
 'epoch': 3.0}

In [30]:
# 使用原来的100个数据进行验证，准确率是0.65,也比原来的高
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.8555477857589722,
 'eval_accuracy': 0.65,
 'eval_runtime': 0.712,
 'eval_samples_per_second': 140.456,
 'eval_steps_per_second': 18.259,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [24]:
trainer.save_model(model_dir)

In [25]:
trainer.save_state()

In [23]:
# trainer.model.save_pretrained("./")

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少